In [ ]:
!git clone https://github.com/markkirby95/LS-Hackathon.git

Cloning into 'LS-Hackathon'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [1]:
# Adding an extra cell

In [2]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

import mlflow


     |████████████████████████████████| 13.9MB 318kB/s 
     |████████████████████████████████| 337kB 48.1MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 1.1MB 48.9MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 163kB 50.0MB/s 
     |████████████████████████████████| 153kB 50.1MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 133kB 43.4MB/s 
     |████████████████████████████████| 2.6MB 51.5MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 204kB 48.6MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 


In [25]:
with mlflow.start_run(run_name="MLflow on Colab"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")

# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background


# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6

from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://8a68ddc05832.ngrok.io


In [7]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

data = load_iris()

feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
y_name = 'target'

data_df = pd.DataFrame(
    data=np.c_[data['data'], data['target']],
    columns=feature_names + [y_name])
data_df.head()


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_df)

In [9]:
x_train = train[feature_names]
y_train = train[y_name]

x_test = test[feature_names]
y_test = test[y_name]

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [11]:
import mlflow
import mlflow.sklearn

In [21]:
mlflow.tracking._DEFAULT_USER_ID = 'mark.kirby'

In [22]:
mlflow.set_tracking_uri("http://localhost:5000")
! mlflow ui

[2020-12-01 22:30:36 +0000] [261] [INFO] Starting gunicorn 20.0.4
[2020-12-01 22:30:36 +0000] [261] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2020-12-01 22:30:36 +0000] [261] [ERROR] Retrying in 1 second.
[2020-12-01 22:30:37 +0000] [261] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2020-12-01 22:30:37 +0000] [261] [ERROR] Retrying in 1 second.
[2020-12-01 22:30:38 +0000] [261] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2020-12-01 22:30:38 +0000] [261] [ERROR] Retrying in 1 second.
[2020-12-01 22:30:39 +0000] [261] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2020-12-01 22:30:39 +0000] [261] [ERROR] Retrying in 1 second.
[2020-12-01 22:30:40 +0000] [261] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2020-12-01 22:30:40 +0000] [261] [ERROR] Retrying in 1 second.
[2020-12-01 22:30:41 +0000] [261] [ERROR] Can't connect to ('127.0.0.1', 5000)
Running the mlflow server failed. Please see the logs above for details.


In [17]:
from sklearn.linear_model import LogisticRegression

tuned_parameters = [
    {'penalty': 'l2', 'solver': 'newton-cg', 'max_iter': 10},
    {'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 10},
    {'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 30},
    {'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 50},
]


cnt = 0 
for param in tuned_parameters:
    with mlflow.start_run():
        lr = LogisticRegression(max_iter=param['max_iter'], penalty=param['penalty'], solver=param['solver'])
        lr.fit(x_train, y_train)
        y_pred = lr.predict(x_test)

        (rmse, mae, r2) = eval_metrics(y_test, y_pred)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_param("max_iter", param['max_iter'])
        mlflow.log_param("penalty", param['penalty'])
        mlflow.log_param("solver", param['solver'])

        mlflow.sklearn.log_model(lr, "model")
        
    cnt += 1

/usr/local/lib/python3.6/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
